In [ ]:

import os
import torch
import data_setup, engine, model, model_utils, visualization
from torchvision import transforms, datasets
#install umap doing: pip install umap-learn

In [ ]:
torch.manual_seed(42)

Hyperparameters

In [ ]:
# Setup hyperparameters
NUM_EPOCHS = 10
INPUT_SIZE = 28*28
NUM_HIDDEN_LAYERS = 2
BATCH_SIZE = 128
HIDDEN_UNITS = 128
LATENT_DIMS = 2
LEARNING_RATE = 0.001
KL_BETA = 1

In [ ]:
# Define the transformations to be applied to the data, normalize between 0 and 1
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,))
])

Define the paths

In [ ]:
# Define the paths for training and testing data
train_path = 'data/mnist/train'
test_path = 'data/mnist/test'


In [ ]:
model_path = "models/"
image_path = "images/"

In [ ]:
# Create directories if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)


In [ ]:
# Download and load the MNIST training dataset
train_dataset = datasets.MNIST(root=train_path, train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root=test_path, train=False, download=True, transform=transform)
print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of test examples: {len(test_dataset)}")
print(f"Dimensions of first image: {train_dataset[0][0].shape}")
print(f"First image: {train_dataset[0][0]}")

In [ ]:
# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_data=train_dataset,
    test_data=test_dataset,
    batch_size=BATCH_SIZE
)

Create model

In [ ]:
# Create model with help from model.py
vae = model.VAE(input_size=INPUT_SIZE,
                  num_hidden_layers=NUM_HIDDEN_LAYERS,
                  hidden_units=HIDDEN_UNITS,
                  latent_dims=LATENT_DIMS).to(device)

In [ ]:
# Set Optimizer
optimizer = torch.optim.Adam(vae.parameters(),
                             lr=LEARNING_RATE)

Train model

In [ ]:
# Start training with help from engine.py
results = engine.train(model=vae,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             beta=KL_BETA,
             epochs=NUM_EPOCHS,
             save_path=model_path,
             device=device)

Save model

In [ ]:
# Save the model with help from utils.py
model_utils.save_model(model=vae,
                 target_dir= model_path,
                 model_name="vae.pth")

Load Model

In [ ]:
# Instantiate a new instance of our model (this will be instantiated with random weights)
loaded_model = model.VAE(input_size=INPUT_SIZE, num_hidden_layers=NUM_HIDDEN_LAYERS, hidden_units=HIDDEN_UNITS, latent_dims=LATENT_DIMS)

# Load the state_dict of our saved model (this will update the new instance of our model with trained weights)
loaded_model.load_state_dict(torch.load(f="{model_path}vae.pth", map_location=torch.device(device)))

In [ ]:
# Plot test images and their reconstructions with help from visualization.py
visualization.plot_reconstructions(model=vae,
                                    test_dataloader=test_dataloader,
                                    device=device,
                                    path = image_path,
                                    n=15)

In [ ]:
# Plot generated images with help from visualization.py
visualization.plot_generated_samples(model=vae,
                              device=device,
                              path = image_path,
                              n=15)

In [ ]:
#Plot training and testing loss curves with help from visualization.py
visualization.plot_loss(train_loss=results["train_loss"],
                        test_loss=results["test_loss"],
                        path= image_path)

In [ ]:
#Plot latent space with help from visualization.py
visualization.plot_latent(model=loaded_model,
                                test_dataloader=test_dataloader,
                                device=device,
                                path = image_path)

In [ ]:
visualization.plot_likelihood_and_kl_loss(train_likelihood=results["train_reconstruction"], train_kl_loss=results["train_kl_loss"], test_kl_loss=results["test_kl_loss"], test_likelihood=results["test_reconstruction"], path= image_path)